# Math 104B: Homework 5
- Name: Eduardo Escoto
- Perm: 7611817
- Email: e_escoto@ucsb.edu
- Instructor: Professor Hector D. Ceniceros
- Term: Winter 2020

In [12]:
from scipy import linalg
import numpy as np

___

### Problem 1

#### Part a)
Implement Gaussian Elimination with partial pivoting to solve a linear system $Ax= b$, given as output the coefficients of a nonsingular, $n\times n$ matrix $A$ and an $n$-vector $b$. Your code should produce the solution $x$ or an error message (if A is non-singular) and the information needed for the $LU$ factorization.

In [13]:
def LU(A, b = None):
    """
    Important: Matrices must be floats, 
    they get casted to integers if not, for some reason?
    """
    n = A.shape[0]
    if b is not None:
        b = b.astype(float)
    U = np.copy(A).astype(float)
    L = np.zeros(A.shape);np.fill_diagonal(L,1)
    P = np.zeros(A.shape);np.fill_diagonal(P,1)
    
    for k in range(n):
        m = np.argmax(abs(U[k:,k])) + k
        
        # Error Checkings
        if np.isclose(U[m, k], 0):
            raise ValueError("Matrix is singular.")

        # Pivoting everything if necessary
        if m != k:
            U[[k,m]] = U[[m, k]]
            P[[k,m]] = P[[m, k]]
            if b is not None:
                b[[k,m]] = b[[m, k]]
        
        # Elimination
        m = np.zeros(n)
        for j in range(k+1, n):
            m[j] = U[j,k]/U[k,k]
            U[j, k:] = U[j, k:] - m[j]*U[k, k:]
            if b is not None:
                b[j] = b[j] - m[j]*b[k]   
        
        # L matrix multiplier placement
        for i in range(n):
            if i > k:
                L[i,k] = m[i]
        
    return (U, L, P, b)

def backSub(A, b):
    n = A.shape[0]
    x = np.zeros(n, dtype=float)
    for k in range(n-1, -1, -1):
        x[k] = (b[k] - np.dot(A[k,k+1:],x[k+1:]))/A[k,k]
    return x

#### Part b)
Let 


\begin{equation}
A = \begin{bmatrix}
5&1&0&2&1 \\
0&4&0&1&2 \\
1&1&4&1&1 \\
0&1&2&6&0 \\
0&0&1&2&4
\end{bmatrix} \quad b = \begin{bmatrix}1\\2\\3\\4\\5\end{bmatrix}
\end{equation}


Use your Gaussian Elimination code to solve $Ax = b$.

##### Solution 
We use our code below!

In [14]:
A = np.array([
    5,1,0,2,1,
    0,4,0,1,2,
    1,1,4,1,1,
    0,1,2,6,0,
    0,0,1,2,4
],dtype = float).reshape(5,5)

b = np.array([1,2,3,4,5], dtype=float).reshape(5,1)

U, L, P, b_piv = LU(A,b)
x = backSub(U,b_piv)

print("L=\r\n", L)
print("U=\r\n", U)
print("x = \r\n", x)

L=
 [[1.         0.         0.         0.         0.        ]
 [0.         1.         0.         0.         0.        ]
 [0.2        0.2        1.         0.         0.        ]
 [0.         0.25       0.5        1.         0.        ]
 [0.         0.         0.25       0.34234234 1.        ]]
U=
 [[ 5.          1.          0.          2.          1.        ]
 [ 0.          4.          0.          1.          2.        ]
 [ 0.          0.          4.          0.4         0.4       ]
 [ 0.          0.          0.          5.55       -0.7       ]
 [ 0.          0.          0.          0.          4.13963964]]
x = 
 [-0.17083787 -0.06746464  0.46028292  0.52448313  0.8726877 ]


In [15]:
P, L, U = scipy.linalg.lu(A)
x = np.linalg.solve(A, b)
print("L = \r\n", L)
print("U = \r\n", U)
print("x = \r\n", x)

L = 
 [[1.         0.         0.         0.         0.        ]
 [0.         1.         0.         0.         0.        ]
 [0.2        0.2        1.         0.         0.        ]
 [0.         0.25       0.5        1.         0.        ]
 [0.         0.         0.25       0.34234234 1.        ]]
U = 
 [[ 5.          1.          0.          2.          1.        ]
 [ 0.          4.          0.          1.          2.        ]
 [ 0.          0.          4.          0.4         0.4       ]
 [ 0.          0.          0.          5.55       -0.7       ]
 [ 0.          0.          0.          0.          4.13963964]]
x = 
 [[-0.17083787]
 [-0.06746464]
 [ 0.46028292]
 [ 0.52448313]
 [ 0.8726877 ]]


#### Part c)
Test your Gaussian Elimination code for


\begin{equation}
A = \begin{bmatrix}
5 & 1 & 0 & 2\\
0 & 4 & 0 & 8\\
1 & 1 & 4 & 2 \\
0 & 1 & 2 & 2
\end{bmatrix}, \quad b = \begin{bmatrix}1\\2\\3\\4\end{bmatrix}
\end{equation}

This Matrix is singular, thus we **SHOULD** get an error.

In [16]:
A = np.array([
    5,1,0,2,
    0,4,0,8,
    1,1,4,2,
    0,1,2,2
],dtype = float).reshape(4,4)
b = np.array([1,2,3,4], dtype=float).reshape(4,1)

U, L , P, b_piv = LU(A,b)
x = backSub(U,b_piv)

ValueError: Matrix is singular.

Checking our solution with `scipy`'s we also get an error.

In [17]:
P, L, U = scipy.linalg.lu(A)
x = np.linalg.solve(A, b)
print("L = \r\n", L)
print("U = \r\n", U)
print("x = \r\n", x)

LinAlgError: Singular matrix

___

### Problem 2

#### Part a)
Let $A$ be an $n \times n$ upper or lower triangular matrix. Prove that the determinant of $A$ is equal to the product of its diagonal entries, i.e. $\det(A) = a_{11}a_{22}\dots a_{nn}$.

###### Solution

To prove this, we will use induction.<br>
**Base Case:** $n = 2$<br>
Let $A$ be a $2\times 2$ upper diagonal matrix.
In this case we have:
$$
A = \begin{bmatrix}
a_{11} & a_{12}\\
0 & a_{22}
\end{bmatrix}
$$

Which then yields the following determinant:
$$\det(A) = (a_{11}a_{22}) - (0 \times a_{12}) = a_{11}a_{22}$$

Notice that the lower, or upper diagonal condition the final answer of $\det(A)$ is the same for both. In the case for lower we have:
$$\det(A) = (a_{11}a_{22}) - (a_{21} \times 0) = a_{11}a_{22}$$

Take the case $n = 3$. 
We have:
$$
A = \begin{bmatrix}
a_{1,1} & a_{1,2} &a_{1, 3}\\
0 & a_{2,2} & a_{2,3}\\
0 & 0& a_{3,3}\\
\end{bmatrix}
$$

Which then yields the following determinant:
\begin{align*}
&\det(A) = a_{1,1}\det\left(\begin{bmatrix}
a_{2,2} &a_{2, 3}\\
0 & a_{3,3}\\
\end{bmatrix}\right) - a_{1,2}\det\left(\begin{bmatrix}
0 & a_{2,3}\\
0 & a_{3,3}
\end{bmatrix}\right) + a_{1,3}\det\left(\begin{bmatrix}
0 & a_{2,2}\\
0 & 0
\end{bmatrix}\right)
\end{align*}

Solving, we have:
$$
\det(A) = a_{1,1}a_{2,2}a_{3,3}
$$
Which are all of the diagonal elements. Notice that eventually, the determinants vanish at the $2\times 2$ level because of the zeros in the first column.

Now asssume that the hypothesis is true for $n = k$<br>
Now take the **Induction Case:** $n = k+1$:<br>
We have now have the matrix $A$ is a $k+1 \times k + 1$ upper triangular matrix.
Thus we have:
\begin{align*}
& \det(A) = a_{11}\det\left(\begin{bmatrix}
a_{2,2} &\dots & \dots &a_{2, k+1}\\
0 & a_{3,3}& \dots& \vdots\\
0 & 0 & \ddots & \vdots \\
0&0&\dots&a_{k+1,k+1}
\end{bmatrix}\right) - a_{12}\det\left(\begin{bmatrix}
0 & a_{2,2}& \dots &a_{2, k+1}\\
0 & 0& \dots& \vdots\\
0 & 0 & \ddots & \vdots \\
0&0&\dots&a_{k+1,k+1}
\end{bmatrix}\right) + \dots \\ & + (-1)^{k+1}a_{k+1,k+1}\left(\begin{bmatrix}
0 & a_{2,2} & \dots &a_{2, k} \\
0 & 0 & \dots & \vdots \\
0 & 0 & \ddots & \vdots \\
0&0&\dots&a_{k,k+1}
\end{bmatrix}\right)
\end{align*}

From the induction hypothesis, we have that the first term is the product of the diagonal, and for the rest of the terms, the first columns are all zeroes. Any matrix with a column of zeroes has a determinant of 0. This is because if a whole column is zeros, then there is a free variable in the system, implying that the matrix is not one-to-one. For a matrix to be invertible, i.e. non-singular, or its determinant not being zero, it must be one-to-one and onto. <br>

Thus we have:

$$\det(A) = a_{1,1}a_{2,2}\dots a_{k+1,k+1} - a_{1,2}\cdot0 + a_{1,3}\cdot0 + \dots + (-1)^{k+1}a_{1,k+1}\cdot0$$
so
$$\det(A) = a_{1,1}a_{2,2}\dots a_{k+1,k+1}$$

Thus, by induction, any lower, or upper diagonal matrixs has a determinant which is the product of its diagonal.

#### Part b) 
Prove that the product of the pivots in the Gaussian Elimination for $Ax = b$ is equal to the determinant of $A$ up to a sign.

###### Solution
Let $P$ be the permutation matrix such that $PA = LU$, and $L,U$ be the lower and upper triangular matrices of the $LU$ decomposition. Additionally, we have that $A$ is a non-singular $n\times n$ matrix.
Now we have:
$$\det(PA) = \det(LU)$$
Which then gives
$$\det(P)\det(A) = \det(L)\det(U)$$

The permutation matrix $P$, is a rearrangement of the rows of the Identity matrix. When switching the rows of any matrix, the determinant only changes in its sign. 

Thus we have:
$$\pm \det(A) = \det(L)\det(U)$$

Now, from part a), we proved that the determinant of any lower, or upper triangular matrix is just the product of its diagonal. By definition, $L$ has a diagonal of only ones, and is lower triangular. Thus, $$\det(L) = 1$$

So now, 
$$\pm \det(A) = \det(U)$$

However, similarly to the determinant of $L$, $U$ is an upper triangular matrix with the pivot points on its diagonal, thus $$\det(U) = a_{1,1}^{(1)}a_{2,2}^{(2)}\dots a_{n,n}^{(n)}$$
which are pivot points. i.e. the $k$-th element on diagonal after the $k$-th pivot.

Thus, 
$$\pm \det(A) = a_{1,1}^{(1)}a_{2,2}^{(2)}\dots a_{n,n}^{(n)}$$

So the product of the pivots is the determinant of $A$ up to a sign.

#### Part c)
Prove that the product of two $n\times n$ lower (upper) triangular matrices is a lower (upper) triangular matrix.

###### Solution 
Let $A$ and $B$ be two $n\times n$ upper triangular matrices. 

We have the $i,j$-th element of $AB$ to be:

$$AB_{ij} = \sum_{k=1}^{n} a_{ik}b_{kj}$$


Notice that since $A$ and $B$ are both upper triangular of the same dimension, we are necessarily below the diagonal when $i > j$, thus whenever we are below the diagonal the value is 0. <br>

Now, let the indices $i,j$ be below the diagonal. This means $i > j$ and the following expression for the sum:

$$AB_{ij} = \sum_{k=1}^{n}a_{ik}b_{kj} + \sum_{k\geq i}a_{ik}b_{kj} = \sum_{k<i}0b_{kj} + \sum_{k \geq i}a_{ik}0 = 0$$

this is because we have $a_{ik} = 0$ when $k < i$. But now, when $k > j$, we have that $b_{kj} = 0$. Now, since in this case we have the condition $i > j$ then we have an overlapping interval such that every value is zero. In other words, there is no range where $i\leq k \leq j$.

Now, let the indices $i,j$ be on, or above the diagonal, i.e., $i \leq j$. We now have:

$$AB_{ij} = \sum_{k= 1}^{n}a_{ik}b_{kj} = \sum_{k< i}0b_{kj} + \sum_{i \leq k \leq j}a_{ik}b_{kj} +\sum_{k \geq j}a_{ik}0 = \sum_{i \leq k \leq j}a_{ik}b_{kj}$$

In this case, since $i \leq j$ there is a range where $i\leq k \leq j$ we have some places that are both being multipled above or on the diagonal, jielding a value.

Thus, we have shown both possible cases. When we are Now, below the diagonal $i > j$, we get $0$, and when we are on or above, $i \leq j$, we yield $\sum_{i \leq k \leq j}a_{ik}b_{kj}$. Thus it is proven.

This can be shown for lower diagonal matrices using the same arguement, but just switching the cases, i.e. we are above the diagonal when $j > i$ and on or below when $i \geq j$

#### Part d)
Let $L_{i}$ be the unit upper triangular matrix that produces the $i$-th elimination step in the Gaussian Elimination algorithm, i.e.

\begin{equation}
L_{i} = \begin{bmatrix}
1 & & & &\\
& \ddots &\cdots &\cdots &&\\
& & 1 & & & \\
& & -m_{i+1,i} & &\\
& & -m_{i+2,i} & \ddots& &\\
& & \vdots & & \ddots&\\
& & -m_{n,i} & & &1\\
\end{bmatrix}
\end{equation}

Prove that
\begin{equation}
L_{i}^{-1} = \begin{bmatrix}
1 & & & &\\
& \ddots &\cdots &\cdots &&\\
& & 1 & & & \\
& & m_{i+1,i} & &\\
& & m_{i+2,i} & \ddots& &\\
& & \vdots & & \ddots&\\
& & m_{m,i} & & &1\\
\end{bmatrix}
\end{equation}

##### Solution
Before we begin, notice that from the values above we can construct the $L$ matrix from the $LU$ decomposition as:

$$L = L^{-1}_{1}L^{-1}_{2}\dots L^{-1}_{n}$$

Thus, for the $i$-th elimination step in the gaussian Elimation algorithm, we have:

$$L_{i}L_{i-1}\dots L_{1}PA = L^{-1}_{i+1}\dots L^{-1}_{n}U$$

At the $i$-th elimation step, each of the values below the diagonal are just subtracting each $m_{i,j}$. Thus to undo, i.e., invert the operation we merely just add the $m_{i,j}$ back.

Additionally, we can prove that $L^{-1}_{i}$ is indeed the inverse of $L_{i}$ if $L^{-1}_{i}L_{i} = I$. Matrix inverses are unique, so if this is indeed the case, then $L^{-1}_{i}$ is the unique inverse.

We have: 

\begin{align*} &
L^{-1}_{i}L_{i} =\begin{bmatrix}
1 & & & &\\
& \ddots &\cdots &\cdots &&\\
& & 1 & & & \\
& & m_{i+1,i} & &\\
& & m_{i+2,i} & \ddots& &\\
& & \vdots & & \ddots&\\
& & m_{m,i} & & &1\\
\end{bmatrix}\cdot\begin{bmatrix}
1 & & & &\\
& \ddots &\cdots &\cdots &&\\
& & 1 & & & \\
& & -m_{i+1,i} & &\\
& & -m_{i+2,i} & \ddots& &\\
& & \vdots & & \ddots&\\
& & -m_{n,i} & & &1\\
\end{bmatrix} = \\
&L^{-1}_{i}L_{i} = \begin{bmatrix}
1 & & & &\\
& \ddots &\cdots &\cdots &&\\
& & 1 & & & \\
& & m_{i+1,i}-m_{i+1,i} & &\\
& & m_{i+2,i}-m_{i+2,i} & \ddots& &\\
& & \vdots & & \ddots&\\
& & m_{i+2,i}-m_{n,i} & & &1\\
\end{bmatrix} = \begin{bmatrix}
1 & & & &\\
& \ddots &\cdots &\cdots &&\\
& & 1 & & & \\
& & 0 & &\\
& & 0 & \ddots& &\\
& & \vdots & & \ddots&\\
& & 0 & & &1\\
\end{bmatrix} = I
\end{align*}

Thus, $L_{i}^{-1}$ is indeed the inverse map of $L_{i}$.

___

### Problem 3
Find an $LU$ factorization of the matrix

$$
A = \begin{bmatrix}
5 & 1 & 0\\
0 & 4 & 0\\
1 & 1 & 4
\end{bmatrix}
$$

##### Solution
To find the $LU$ Factorization, we can step through the Gaussian Elimination with pivoting, manually.

$$
A = A^{(0)} = \begin{bmatrix}
5 & 1 & 0\\
0 & 4 & 0\\
1 & 1 & 4
\end{bmatrix} \implies L_{1} = \begin{bmatrix}
1 & 0 & 0\\
0 & 1 & 0\\
\frac{1}{5} & 0 & 1
\end{bmatrix}$$

Then, 
$$
L_{1}^{-1}A = A^{(1)} = \begin{bmatrix}
5 & 1 & 0\\
0 & 4 & 0\\
0 & \frac{4}{5} & 4
\end{bmatrix} \implies L_{2} = \begin{bmatrix}
1 & 0 & 0\\
0 & 1 & 0\\
0 & \frac{1}{5} & 1
\end{bmatrix}
$$
Finally,

$$
L_{2}^{-1}L_{1}^{-1}A = A^{(2)} =\begin{bmatrix}
5 & 1 & 0\\
0 & 4 & 0\\
0 & 0 & 4
\end{bmatrix} \implies L_{3} = \begin{bmatrix}
1 & 0 & 0\\
0 & 1 & 0\\
0 & 0 & 1
\end{bmatrix}
$$

So finally, 

$$
L_{3}^{-1}L_{2}^{-1}L_{1}^{-1}A = A^{(3)} = U = \begin{bmatrix}
5 & 1 & 0\\
0 & 4 & 0\\
0 & 0 & 4
\end{bmatrix}
$$
And gives 
$$
L = L_{1}L_{2}L_{3} = \begin{bmatrix}
1 & 0 & 0\\
0 & 1 & 0\\
\frac{1}{5} & \frac{1}{5} & 1
\end{bmatrix}
$$

Notice that we made no pivots, thus we have $A = LU$, i.e. $P = I$.

Checking our solution:
$$
LU = \begin{bmatrix}
1 & 0 & 0\\
0 & 1 & 0\\
\frac{1}{5} & \frac{1}{5} & 1
\end{bmatrix}\begin{bmatrix}
5 & 1 & 0\\
0 & 4 & 0\\
0 & 0 & 4
\end{bmatrix} = \begin{bmatrix}
5 & 1 & 0\\
0 & 4 & 0\\
5\times\frac{1}{5} & \frac{1}{5} + 4\times\frac{1}{5} & 4
\end{bmatrix} = \begin{bmatrix}
5 & 1 & 0\\
0 & 4 & 0\\
1 & 1 & 4
\end{bmatrix} = A
$$

Thus, we arrived at the correct $LU$ factorization.

Additionally, we use our code to check our solution!

In [18]:
A = np.array([
    5,1,0,
    0,4,0,
    1,1,4
], dtype = float).reshape((3,3))
U, L, P, b = LU(A)

print("L=\r\n", L)
print("U=\r\n", U)

L=
 [[1.  0.  0. ]
 [0.  1.  0. ]
 [0.2 0.2 1. ]]
U=
 [[5. 1. 0.]
 [0. 4. 0.]
 [0. 0. 4.]]


We can also further check with `scipy`'s solution:

In [19]:
P, L, U = scipy.linalg.lu(A)
print("L=\r\n", L)
print("U=\r\n", U)

L=
 [[1.  0.  0. ]
 [0.  1.  0. ]
 [0.2 0.2 1. ]]
U=
 [[5. 1. 0.]
 [0. 4. 0.]
 [0. 0. 4.]]


___

### Problem 4
Find the Choleski factorization $A = LL^{T}$

$$
A = \begin{bmatrix}
3 & -1 & 0\\
-1 & 3 & -1\\
0 & -1 & 3
\end{bmatrix}
$$

##### Solution
In order to compute the Choleski Factorization, we can step through the algorithm manually.

**i = 1**:<br>
$l_{11} = \sqrt{a_{11}} = \sqrt{3}$

**i = 1, j = 2**:<br>
$l_{21} = \frac{a_{12}}{l_{11}} = \frac{-1}{\sqrt{3}}$

**i = 1, j = 2**:<br>
$l_{31} = \frac{a_{13}}{l_{11}} = 0$

Thus, so far we have:

$$
L = \begin{bmatrix}
\sqrt{3} & 0 & 0\\
\frac{-1}{\sqrt{3}} & 0 & 0\\
0 & 0 & 0
\end{bmatrix}
$$

Next iteration:
**i = 2**:<br>
$l_{22} = \sqrt{a_{11}-l_{21}^{2}} = \sqrt{3-\left(\frac{-1}{\sqrt{3}}\right)^{2}} = \sqrt{\frac{8}{3}}$

**i = 2, j = 3**:<br>
$l_{32} = \frac{a_{12}-l_{21}l_{31}}{l_{22}} = \frac{-1}{\sqrt{\frac{8}{3}}} = -\sqrt{\frac{3}{8}}$

Then so far we have:

$$
L = \begin{bmatrix}
\sqrt{3} & 0 & 0\\
\frac{-1}{\sqrt{3}} & \sqrt{\frac{8}{3}} & 0\\
0 & -\sqrt{\frac{3}{8}} & 0
\end{bmatrix}
$$

Finally, we have the last iteration:
**i = 3**:<br>
$l_{33} = \sqrt{a_{33}-l_{31}^{2}-l_{32}^{2}} = \sqrt{3- 0 -\left(\sqrt{\frac{3}{8}}\right)^{2}} = \sqrt{\frac{21}{3}}$

Which gives our solution of:

$$
L = \begin{bmatrix}
\sqrt{3} & 0 & 0\\
\frac{-1}{\sqrt{3}} & \sqrt{\frac{8}{3}} & 0\\
0 & -\sqrt{\frac{3}{8}} & \sqrt{\frac{21}{3}}
\end{bmatrix}
$$

Below, we check our solution:

In [20]:
L = np.array([
    np.sqrt(3), 0, 0,
    -1/np.sqrt(3), np.sqrt(8/3),0,
    0, -np.sqrt(3/8), np.sqrt(21/8)
]).reshape((3,3))

print("L = \r\n", L)
print("L^T =  \r\n",L.T)
print("LL^T =  \r\n", np.matmul(L,L.T))

L = 
 [[ 1.73205081  0.          0.        ]
 [-0.57735027  1.63299316  0.        ]
 [ 0.         -0.61237244  1.62018517]]
L^T =  
 [[ 1.73205081 -0.57735027  0.        ]
 [ 0.          1.63299316 -0.61237244]
 [ 0.          0.          1.62018517]]
LL^T =  
 [[ 3. -1.  0.]
 [-1.  3. -1.]
 [ 0. -1.  3.]]


Now checking our solution with `scipy`'s Choleski factorization function:

In [21]:
A = np.array([
    3,-1,0,
    -1,3,-1,
    0,-1,3
], dtype=float).reshape((3,3))

print("L = \r\n", scipy.linalg.cholesky(A).T)
print("L^T = \r\n", scipy.linalg.cholesky(A))
print("LL^T = \r\n", np.matmul(scipy.linalg.cholesky(A).T, scipy.linalg.cholesky(A)))

L = 
 [[ 1.73205081  0.          0.        ]
 [-0.57735027  1.63299316  0.        ]
 [ 0.         -0.61237244  1.62018517]]
L^T = 
 [[ 1.73205081 -0.57735027  0.        ]
 [ 0.          1.63299316 -0.61237244]
 [ 0.          0.          1.62018517]]
LL^T = 
 [[ 3. -1.  0.]
 [-1.  3. -1.]
 [ 0. -1.  3.]]
